In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from models.pruned_my_tinyyolo2 import PrunedMyTinyYoloV2
from models.tinyyolov2 import TinyYoloV2
from models.my_tinyyolov2 import MyTinyYoloV2
from utils.loss import YoloLoss
from utils.ap import ap#, precision_recall_levels, preprocess_for_ap
from utils.train import validate
from utils.dataloader import VOCDataLoaderPerson
import seaborn as sns
import pandas as pd

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

/home/artemis/.local/lib/python3.12/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/artemis/.local/lib/python3.12/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:



state_dict = torch.load('models/configs/voc_fused_10.pt')
#state_dict = torch.load('results/voc_finetuned_123_frozen_layers_300_epochs_00003_lr_0001_decay.pt')

net = MyTinyYoloV2(num_classes=1)
net.load_state_dict(state_dict)


times, losses = [], []

criterion = YoloLoss(anchors=net.anchors)

frozen_layers = []
nr_frozen = len(frozen_layers)


for key, param in net.named_parameters():
    if any(x in key for x in frozen_layers):
        param.requires_grad = False
    
optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, net.parameters()))

step_lengths = [0.5, 0.2, 0.1, 0.05, 0.025]

max_acc = -1
max_filter = 0.5
max_nms = 0.5

for sl in range(1, len(step_lengths)):
    step_length = step_lengths[sl]
    prev_step_length = step_lengths[sl - 1]
    
    start_nms = max_nms - prev_step_length
    end_nms = max_nms + prev_step_length
    nms_threshold_steps = int((end_nms - start_nms) / step_length + 1)

    if (start_nms < 0):
        end_nms -= start_nms
        start_nms = 0
        
    if (end_nms > 1):
        start_nms -= (end_nms - 1)
        end_nms = 1
    
    start_filter = max_filter - prev_step_length
    end_filter = max_filter + prev_step_length
    filter_threshold_steps = int((end_filter - start_filter) / step_length + 1)
    
    if (start_filter < 0):
        end_filter -= start_filter
        start_filter = 0
        
    if (end_filter > 1):
        start_filter -= (end_filter + 1)
        end_filter = 1
    
    nms_thresholds = torch.linspace(start_nms, end_nms, steps=nms_threshold_steps)
    filter_thresholds = torch.linspace(start_filter, end_filter, steps=filter_threshold_steps)
    
    results = torch.zeros([filter_threshold_steps, nms_threshold_steps])
    
    max_acc = 0
    max_filter = 0
    max_nms = 0
    
    
    for i in range(len(filter_thresholds)):
        filter_threshold = filter_thresholds[i]
        #print(f"Filter Threshold: {filter_threshold}")
        for j in range(len(nms_thresholds)):
            nms_threshold = nms_thresholds[j]
            #print(f"NMS Threshold: {nms_threshold}")
    
            p, r, l, t = validate(net, VOCDataLoaderPerson(train=False, batch_size=1, shuffle=False), optimizer, criterion, device=device, filter_threshold=filter_threshold, nms_threshold=nms_threshold, batches=1000)
    
            acc = ap(p, r)
            
            
            #print('average time', t)
            #print('average test losses', l)
            #print('average precision', acc)
            losses.append(l)
            times.append(t)
    
            if (acc > max_acc):
                max_acc = acc
                max_filter = filter_threshold
                max_nms = nms_threshold
    
            results[i][j] = acc
            #print("---------------------------------------------------------")
    
    print('Max accuracy:', max_acc)
    print('With filter threshold:', max_filter)
    print('With NMS threshold:', max_nms)
    cm = sns.light_palette("blue", as_cmap=True)
    x=pd.DataFrame(torch.round(100 * results, decimals=2))
    x=x.style.background_gradient(cmap=cm)
    display(x)


------------------------Validation-----------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 75.10it/s]


------------------------Validation-----------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:37<00:00, 26.96it/s]


------------------------Validation-----------------------


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.08it/s]


------------------------Validation-----------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 85.38it/s]

Max accuracy: 0.571006917991467
With filter threshold: tensor(0.0500)
With NMS threshold: tensor(0.5250)


,0,1,2,3,4
0,53.259998,55.169998,54.830002,54.450001,54.639999
1,53.919998,55.810001,55.439999,55.040001,55.200001
2,53.169998,57.099998,56.639999,56.139999,56.209999
3,53.169998,55.090000,54.750000,57.060001,57.090000
4,53.169998,55.090000,54.750000,54.380001,54.580002


In [ ]:
#plt.imshow(results.numpy())
plt.matshow(results.numpy())
plt.colorbar()
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd

cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(torch.round(100 * results, decimals=2))
x=x.style.background_gradient(cmap=cm)
display(x)

In [ ]:
from utils.utils import identify_threshold

max_accs = torch.zeros(8)
max_nms = torch.zeros_like(max_accs)
max_filter = torch.zeros_like(max_accs)

for i in range(8):
    sd = torch.load(f"results/voc_pruned_{i * 5}_7_frozen_layers_300_epochs_00003_lr_001_decay.pt")

    max_accs[i], max_nms[i], max_filter[i] = identify_threshold(PrunedMyTinyYoloV2, sd, device=device)


print('Accuracies:', max_accs)
print('With filter thresholds:', max_filter)
print('With NMS thresholds:', max_nms)

------------------------Validation-----------------------


  0%|                                                                                                          | 0/1000 [00:00<?, ?it/s]/home/artemis/.local/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 107.41it/s]


------------------------Validation-----------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.78it/s]


------------------------Validation-----------------------


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:22<00:00,  7.00it/s]


------------------------Validation-----------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:47<00:00,  5.98it/s]


------------------------Validation-----------------------


 37%|███████████████████████████████████▌                                                            | 370/1000 [01:28<02:20,  4.49it/s]